In [ ]:
# If gspx is not installed, we add it to the path
import os, sys
gdir = os.path.dirname(os.getcwd())  # parent folder
sys.path.insert(0, gdir)

In [ ]:
import numpy as np

from gspx.utils.display import plot_graph
from gspx.datasets import WeatherGraphData, uk_weather
from gspx.signals import QuaternionSignal
from gspx.qgsp import create_quaternion_weights, QGFT, QMatrix

uk_data = WeatherGraphData()
Ar, coords = uk_data.graph
s = uk_data.signal

df = uk_weather()

Aq = create_quaternion_weights(
    Ar, df, icols=['humidity'], jcols=['temp'],
    kcols=['wind_speed'])

plot_graph(
    Aq.abs(), coords=coords,
    figsize=(2, 4), colormap='viridis',
    node_size=40)

In [ ]:
from gspx.utils.quaternion_matrix import implode_quaternions
from gspx.qgsp import QMatrix

def qinv(Q, rtol=1e-05, atol=1e-08):
    ca = Q.complex_adjoint
    assert np.linalg.cond(ca) < 1/sys.float_info.epsilon, (
        "The given matrix is not invertible."
    )
    nrows, _ = ca.shape
    n = int(nrows/2)
    ca_inv = np.linalg.inv(ca)
    inv00 = ca_inv[:n, :n]
    inv01 = ca_inv[:n, n:]
    inv10 = ca_inv[n:, :n]
    inv11 = ca_inv[n:, n:]
    assert np.allclose(inv00, inv11.conjugate(), rtol=rtol, atol=atol)
    assert np.allclose(inv01, - inv10.conjugate(), rtol=rtol, atol=atol)
    qmatrix = implode_quaternions(np.dstack((
        np.real(inv00),
        np.imag(inv00),
        np.real(inv01),
        np.imag(inv01),
    )))
    return QMatrix.from_matrix(qmatrix)

In [ ]:
QMatrix.visualize((qinv(Aq) * Aq))